In [3]:
#Importing essential libraries
import pennylane as qml
from pennylane import numpy as np

#Initialising device
device = qml.device('default.qubit', wires = 4)

@qml.qnode(device, diff_method = "backprop")
def circuit(params):
  qml.StronglyEntanglingLayers(params, wires = [0, 1, 2, 3])
  return qml.expval(qml.PauliZ(0) @ qml.PauliZ(1) @ qml.PauliZ(2) @ qml.PauliZ(3))

#initialize circuit parameters
param_shape = qml.StronglyEntanglingLayers.shape(n_wires = 4, n_layers = 15)
params = np.random.normal(scale = 0.1, size = param_shape, requires_grad = True)
print(circuit(params))

0.899066051888016


In [5]:
#Calculating time needed to perform a forward pass
import timeit

reps = 3
num = 10

times = timeit.repeat("circuit(params)", globals = globals(), number = num, repeat = reps)
forward_time = min(times)/num
print(f"Forward pass (best of {reps}) : {forward_time} sec per loop")

Forward pass (best of 3) : 0.07029512450000083 sec per loop


In [6]:
#Calculating time needed to perform backpropagation
times = timeit.repeat("qml.grad(circuit)(params)", globals = globals(), number = num, repeat = reps)
backward_time = min(times)/num
print(f"Backward pass (best of {reps}): {backward_time} per loop")

Backward pass (best of 3): 0.1335601462999989 per loop


In [ ]:
#Comparing time taken by backpropagation and parameter shift rule

reps = 2
num = 3

forward_shift = []
gradient_shift = []
forward_backprop = []
gradient_backprop = []

for depth in range(0, 21):
  param_shape = qml.StronlyEntanglingLayers.shape(n_wires = 4, n_layers = depth)
  params = np.random.normal(scale = 0.1, size = param_shape, requires_grad = True)
  num_params = params.size

  # forward pass timing
  # ===================

  qnode_shift = qml.Qnode(circuit, device, diff_method = "parameter-shift")
  qnode_backprop = qml.Qnode(circuit, device, diff_method = "backprop")

  #parameter - shift
  t = timeit.repeat("qnode_shift(params)", globals = globals(), number = num, repeat = reps)
  forward_shift.append([num_params, min(t)/num])

  #backpropagation
  t = timeit.repeat("qnode_backprop(params)", globals = globals(), number = num, repeat = reps)
  forward_backprop.append([num_params, min(t)/num])

  if num_params == 0:
    continue

  #Gradient timing
  # ==================

  qnode_shift = qml.qnode(circuit, device, diff_method = "parameter-shift")
  qnode_backprop = qml.qnode(circuit, device, diff_method = "backprop")

  #parameter shift
  t = timeit.repeat("qml.grad(qnode_shift)(params)", globals = globals(), number = num, repeat = reps)
  gradient_shift.append([num_params, min(t)/num])

  #backpropagation
  t = timeit.repeat("qml.grad(qnode_backprop)(params)", globals = globals(), number = num, repeat = reps)
  gradient_backprop.append([num_params, min(t)/num])

  gradient_shift = np.array(gradient_shift).T
  forward_shift = np.array(forward_shift).T
  gradient_backprop = np.array(gradient_backdrop).T
  forward_backdrop = np.array(forward_backdrop).T